In [13]:
import os
import json
import traceback
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
from typing import Optional

# -----------------------
# Config
# -----------------------
load_dotenv()
#openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o-mini"  # or "gpt-4" for max reliability
MAX_ITERS = 8

# -----------------------
# Tools
# -----------------------
def safe_exec(code: str, df: pd.DataFrame):
    """Safely execute Python code with pandas/numpy context."""
    safe_globals = {"pd": pd, "np": np, "df": df, "__builtins__": {}}
    safe_globals["len"] = len
    safe_globals["round"] = round
    safe_globals["sum"] = sum
    safe_globals["min"] = min
    safe_globals["max"] = max
    safe_globals["abs"] = abs

    try:
        local_vars = {}
        exec(code, safe_globals, local_vars)
        result = local_vars.get("result", None)
        if result is None:
            return "Error: Your code must assign the final output to a variable called 'result'."
        if isinstance(result, pd.DataFrame):
            return f"DataFrame {result.shape}\n{result.head(5).to_string()}", result
        elif isinstance(result, pd.Series):
            return f"Series {result.shape}\n{result.head(5).to_string()}", result
        else:
            return str(result), result
    except Exception as e:
        return f"Error: {e}\n{traceback.format_exc()}", None

def plot_tool(result, spec: dict):
    """Plot a pandas result based on JSON spec {type, x, y}."""
    try:
        kind = spec.get("type", "bar")
        x = spec.get("x")
        y = spec.get("y")
        title = spec.get("title", "Plot")

        plt.clf()
        if isinstance(result, pd.Series):
            result.plot(kind=kind)
        elif isinstance(result, pd.DataFrame):
            if x and y:
                result.plot(kind=kind, x=x, y=y)
            else:
                result.plot(kind=kind)
        else:
            return "Plotting requires a pandas Series or DataFrame."

        plt.title(title)
        out_path = "plot.png"
        plt.savefig(out_path)
        plt.close()
        return f"Plot saved to {out_path}"
    except Exception as e:
        return f"Plot error: {e}"

TOOLS = {"python_repl": safe_exec, "plot_tool": plot_tool}

# -----------------------
# Pydantic Schema
# -----------------------
class AgentStep(BaseModel):
    Thought: Optional[str]
    Action: Optional[str]
    ActionInput: Optional[str]
    FinalAnswer: Optional[str]

# -----------------------
# System Prompt
# -----------------------
PANDAS_GUIDE = """
You are working with a pandas DataFrame called `df`. 
Useful pandas/numpy syntax:
- df['col'].mean(), df['col'].sum(), df['col'].unique()
- df.groupby('col')['val'].mean().reset_index()
- df[(df['col'] == 'value') & (df['other'] > 10)]
- df.sort_values('col', ascending=False).head(5)
- df['col'].value_counts()
- df.pivot_table(index='col1', columns='col2', values='val', aggfunc='sum')
- np.round(), np.mean(), np.median(), np.std()
Always assign the final result to a variable called `result`.
"""

SYSTEM_PROMPT = f"""
You are a robust CSV QA assistant. 
You must always return **valid JSON** matching this schema:

{{
  "Thought": "your reasoning",
  "Action": "python_repl" or "plot_tool" or null,
  "ActionInput": "the code string (if python_repl) or JSON string (if plot_tool)",
  "FinalAnswer": "final natural language answer or null"
}}

Rules:
- Use "python_repl" when you need to compute with pandas/numpy. 
- Always assign the final pandas/numpy result to a variable called `result`.
- Use "plot_tool" when visualization is requested. Its ActionInput must be JSON with keys: type, x, y, title.
- If done, set FinalAnswer, and Action/ActionInput=null.
- Never invent tools.
- Only return JSON. No extra text outside JSON.

Pandas quick reference:
{PANDAS_GUIDE}
"""

# -----------------------
# ReAct Loop
# -----------------------
def run_agent(df: pd.DataFrame, question: str, max_iters=MAX_ITERS):
    conversation = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Question: {question}\nColumns: {list(df.columns)}"}
    ]
    last_result = None

    for _ in range(max_iters):
        resp = client.chat.completions.create(
            model=MODEL,
            messages=conversation,
            temperature=0
        )
        #content = resp["choices"][0]["message"]["content"]
        content =resp.choices[0].message.content
        try:
            step = AgentStep.parse_raw(content)
        except ValidationError:
            conversation.append({"role": "user", "content": "Invalid JSON. Retry with valid schema."})
            continue

        print("Thought:", step.Thought)

        if step.FinalAnswer:
            return step.FinalAnswer

        observation = ""
        if step.Action == "python_repl":
            observation, last_result = TOOLS["python_repl"](step.ActionInput, df)
        elif step.Action == "plot_tool":
            if last_result is None:
                observation = "No previous result to plot."
            else:
                try:
                    spec = json.loads(step.ActionInput)
                    observation = TOOLS["plot_tool"](last_result, spec)
                except Exception as e:
                    observation = f"Invalid plot spec: {e}"
        else:
            observation = "No valid action."

        conversation.append({"role": "assistant", "content": content})
        conversation.append({"role": "user", "content": f"Observation: {observation}"})

    return "Max iterations reached without Final Answer."


In [14]:
df = pd.DataFrame({
    "Year": [2022, 2022, 2023, 2023],
    "Region": ["US", "EU", "US", "EU"],
    "Revenue": [100, 120, 150, 170],
    "Units": [10, 12, 15, 17]
})

answer = run_agent(df, "whICH REGion have highest revenue")
print("\nFinal Answer:", answer)

Thought: To find the region with the highest revenue, I will group the data by 'Region' and sum the 'Revenue' for each region, then identify the region with the maximum revenue.
Thought: The observation indicates that the EU region has the highest revenue. I will confirm this by calculating the total revenue for each region and identifying the maximum.
Thought: The DataFrame confirms that the EU region has the highest revenue of 290. I will finalize the answer based on this observation.

Final Answer: The region with the highest revenue is the EU, with a total revenue of 290.


In [5]:
print(openai.__version__)

1.45.0
